In [1]:
from preprocess_income_data import preprocess_data

X_train, X_test, X_val, y_train, y_val, y_test = preprocess_data()

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from const import N_SPLITS, RANDOM_STATE, KNN, KNN_ID

ALGORITHM_NAME = KNN
ALGORITHM_ID = KNN_ID


kf = KFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

n_neighbors = [x+1 for x in range(30)]

steps = [(ALGORITHM_NAME, KNeighborsClassifier())]
pipeline = Pipeline(steps)

parameters = {
    f"{ALGORITHM_NAME}__n_neighbors": n_neighbors,
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=kf, scoring="accuracy")

cv.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20, 21, 22, 23, 24, 25, 26, 27,
                                              28, 29, 30]},
             scoring='accuracy')

In [3]:
cv.best_score_

0.8421424975820142

In [4]:
from process_cv_results import process_cv_scores, process_best_params, get_classification_metrics, results_to_csv
accuracy_scores_df = process_cv_scores(cv, algorithm_id=ALGORITHM_ID)
accuracy_scores_df.head()


,algorithm_id,accuracy_score
0,1,0.804128
1,1,0.822646
2,1,0.826792
3,1,0.832965
4,1,0.829924


In [5]:
best_params_df = process_best_params(
    cv, algorithm_id=ALGORITHM_ID, algorithm_name=ALGORITHM_NAME)
best_params_df.head()


,algorithm_id,hyperparameter_name,best_value
0,1,n_neighbors,18


In [6]:
y_pred = cv.predict(X_test)
classification_metrics_df = get_classification_metrics(
    y_test, y_pred, cv, ALGORITHM_ID, ALGORITHM_NAME)

classification_metrics_df.head()


[[5798  446]
 [ 812 1084]]

---


,algorithm_id,algorithm_name,best_accuracy,precision,recall,roc_auc_score,true_positives,false_positives,false_negatives,true_negatives
0,1,knn,0.842142,0.708497,0.57173,0.750151,5798,446,812,1084


In [7]:
dfs = [accuracy_scores_df, best_params_df, classification_metrics_df]

results_to_csv(dfs, ALGORITHM_ID)
